In [0]:
## S0 前提事項

- アカウントレベルにて次のグループを作成して、それぞれのグループに所属したユーザーにて Databricks Workspace にログインしてセルを実行する。
    - uc_test__uc_test_01__admin -> アカウント管理者グループ
    -　uc_test_all_read-> 権限付与対象者グループ
- 本ノートブックは、`uc_test__uc_test_01__admin`の実行を想定している。

In [0]:
## S1 カタログ、スキーマ、テーブルの検証

In [0]:
### S1-1 カタログ、スキーマ、テーブルの作成

In [0]:
%sql
-- カタログの作成
CREATE CATALOG IF NOT EXISTS uc_test_01;
ALTER CATALOG uc_test_01 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
DESCRIBE CATALOG uc_test_01

info_name,info_value
Catalog Name,uc_test_01
Comment,
Owner,uc_test__uc_test_01__admin
Catalog Type,Regular


In [0]:
%sql
-- スキーマ作成
CREATE SCHEMA IF NOT EXISTS uc_test_01.schema_01;
ALTER SCHEMA uc_test_01.schema_01 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
DESCRIBE SCHEMA uc_test_01.schema_01

database_description_item,database_description_value
Catalog Name,uc_test_01
Namespace Name,schema_01
Comment,
Location,
Owner,uc_test__uc_test_01__admin


In [0]:
%sql
-- テーブル作成
CREATE
OR REPLACE table uc_test_01.schema_01.table_01 (
  str_col string,
  int_col int,
  date_col date
);
ALTER TABLE
  uc_test_01.schema_01.table_01 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
DESCRIBE TABLE uc_test_01.schema_01.table_01

col_name,data_type,comment
str_col,string,null
int_col,int,null
date_col,date,null


In [0]:

### S1-2 スキーマのみへの権限付与

In [0]:
%sql
GRANT USE SCHEMA ON SCHEMA uc_test_01.schema_01 TO uc_test_all_read;
GRANT
SELECT
  ON SCHEMA uc_test_01.schema_01 TO uc_test_all_read;

In [0]:
%sql
SHOW GRANT ON CATALOG uc_test_01

Principal,ActionType,ObjectType,ObjectKey


In [0]:
%sql
SHOW GRANT ON SCHEMA uc_test_01.schema_01

Principal,ActionType,ObjectType,ObjectKey
uc_test_all_read,SELECT,SCHEMA,uc_test_01.schema_01
uc_test_all_read,USE SCHEMA,SCHEMA,uc_test_01.schema_01


In [0]:
### S1-3 カタログへの権限付与

In [0]:
%sql
GRANT USE CATALOG ON CATALOG uc_test_01 TO uc_test_all_read;

In [0]:
%sql
SHOW GRANT ON CATALOG uc_test_01

Principal,ActionType,ObjectType,ObjectKey
uc_test_all_read,USE CATALOG,CATALOG,uc_test_01


In [0]:
## S2 関数と Volumes の検証

In [0]:
### S2-1 関数の作成

In [0]:
%sql
CREATE
OR REPLACE FUNCTION uc_test_01.schema_01.get_rand_num() RETURNS float NOT DETERMINISTIC CONTAINS SQL RETURN rand();

In [0]:
%sql
SELECT
  uc_test_01.schema_01.get_rand_num()

uc_test_01.schema_01.get_rand_num()
0.17621483


In [0]:
%sql
DESCRIBE FUNCTION uc_test_01.schema_01.get_rand_num;

function_desc
Function: uc_test_01.schema_01.get_rand_num
Type: SCALAR
Input: ()
Returns: FLOAT


In [0]:
# S2-2 関数への権限付与

In [0]:
%sql
GRANT EXECUTE ON CATALOG uc_test_01 TO uc_test_all_read;

In [0]:
%sql
SHOW GRANT ON SCHEMA uc_test_01.schema_01

Principal,ActionType,ObjectType,ObjectKey
uc_test_all_read,SELECT,SCHEMA,uc_test_01.schema_01
uc_test_all_read,USE SCHEMA,SCHEMA,uc_test_01.schema_01
uc_test_all_read,EXECUTE,CATALOG,uc_test_01


In [0]:
# S2-3 Volumes の作成

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS uc_test_01.schema_01.volumes_01;
ALTER VOLUME uc_test_01.schema_01.volumes_01 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
CREATE
OR REPLACE TABLE delta.`/Volumes/uc_test_01/schema_01/volumes_01/delta_01` AS
SELECT
  "abc" AS string_col,
  1 :: int AS int_col,
  CAST('2020-01-01' AS date) AS date_col

num_affected_rows,num_inserted_rows


In [0]:
%sql
list '/Volumes/uc_test_01/schema_01/volumes_01/delta_01'

path,name,size,modification_time
/Volumes/uc_test_01/schema_01/volumes_01/delta_01/_delta_log/,_delta_log/,0,1692082822000
/Volumes/uc_test_01/schema_01/volumes_01/delta_01/part-00000-12f392ce-cfc4-44cc-bfa5-39ed07b5e405.c000.snappy.parquet,part-00000-12f392ce-cfc4-44cc-bfa5-39ed07b5e405.c000.snappy.parquet,894,1692082825000


In [0]:
%sql
-- ストレージ名が表示されてしまうため、一時的にコメントアウト
-- DESCRIBE VOLUME uc_test_01.schema_01.volumes_01

In [0]:
%sql
-- ストレージ名が表示されてしまうため、一時的にコメントアウト
-- DESCRIBE DETAIL delta.`/Volumes/uc_test_01/schema_01/volumes_01/delta_01`;

In [0]:
# S2-4 Volumes への権限付与

In [0]:
%sql
GRANT READ VOLUME ON SCHEMA uc_test_01.schema_01 TO uc_test_all_read;

In [0]:
# S3 動的ビューの検証

In [0]:
# S3-0 事前準備

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS uc_test_02;
ALTER CATALOG uc_test_02 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS uc_test_02.schema_01;
ALTER SCHEMA uc_test_02.schema_01 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
CREATE TABLE uc_test_02.schema_01.src_table AS
SELECT
  "A" AS COMPANY_CODE,
  "A会社" AS COMPANY_NAME,
  100 AS SALES_AMOUNT
UNION ALL
SELECT
  "B" AS COMPANY_CODE,
  "B会社" AS COMPANY_NAME,
  200 AS SALES_AMOUNT
UNION ALL
SELECT
  "C" AS COMPANY_CODE,
  "C会社" AS COMPANY_NAME,
  300 AS SALES_AMOUNT

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  *
FROM
  uc_test_02.schema_01.src_table

COMPANY_CODE,COMPANY_NAME,SALES_AMOUNT
B,B会社,200
A,A会社,100
C,C会社,300


In [0]:
%sql
-- 動的ビューを配置するスキーマを作成
CREATE SCHEMA IF NOT EXISTS uc_test_02.schema_dyanamic_view;
ALTER SCHEMA uc_test_02.schema_dyanamic_view OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
-- 動的ビューを配置するスキーマへの権限を付与
GRANT USE CATALOG ON CATALOG uc_test_02 TO uc_test_all_read;
GRANT USE SCHEMA ON SCHEMA uc_test_02.schema_dyanamic_view TO uc_test_all_read;
GRANT
SELECT
  ON SCHEMA uc_test_02.schema_dyanamic_view TO uc_test_all_read;

In [0]:
### S3-1 列レベルのアクセス許可

In [0]:
%sql
CREATE
OR REPLACE VIEW uc_test_02.schema_dyanamic_view.column_level_permissions AS
SELECT
  CASE
    WHEN is_account_group_member('uc_test__uc_test_01__admin') THEN COMPANY_CODE
    WHEN is_account_group_member('uc_test_all_read')
    AND COMPANY_CODE = "A" THEN COMPANY_CODE
    ELSE 'REDACTED'
  END AS COMPANY_CODE,
  CASE
    WHEN is_account_group_member('uc_test__uc_test_01__admin') THEN COMPANY_NAME
    WHEN is_account_group_member('uc_test_all_read')
    AND COMPANY_CODE = "A" THEN COMPANY_NAME
    ELSE 'REDACTED'
  END AS COMPANY_NAME,
  SALES_AMOUNT
FROM
  uc_test_02.schema_01.src_table

In [0]:
%sql
SELECT
  *
FROM
  uc_test_02.schema_dyanamic_view.column_level_permissions

COMPANY_CODE,COMPANY_NAME,SALES_AMOUNT
B,B会社,200
A,A会社,100
C,C会社,300


In [0]:
### S3-2 行レベルのアクセス許可

In [0]:
%sql
CREATE VIEW uc_test_02.schema_dyanamic_view.row_level_permissions AS
SELECT
  COMPANY_CODE,
  COMPANY_NAME,
  SALES_AMOUNT
FROM
  uc_test_02.schema_01.src_table
WHERE
  CASE
    WHEN is_account_group_member('uc_test__uc_test_01__admin') THEN TRUE
    WHEN is_account_group_member('uc_test_all_read')
    AND COMPANY_CODE = "A" THEN TRUE
    ELSE FALSE
  END;

In [0]:
%sql
SELECT
  *
FROM
  uc_test_02.schema_dyanamic_view.row_level_permissions

COMPANY_CODE,COMPANY_NAME,SALES_AMOUNT
B,B会社,200
A,A会社,100
C,C会社,300


In [0]:
## S4 ユースケース別ワークフローの検証

In [0]:
### S4-1 カタログの払出検証

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS uc_test_50

In [0]:
%sql
-- 権原付与
ALTER CATALOG uc_test_50 OWNER TO uc_test_all_read;

In [0]:
%sql
DESCRIBE CATALOG uc_test_50

info_name,info_value
Catalog Name,uc_test_50
Comment,
Owner,uc_test_all_read
Catalog Type,Regular


In [0]:
### S4-2 スキーマの払出検証

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS uc_test_51;
ALTER CATALOG uc_test_51 OWNER TO uc_test__uc_test_01__admin;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS uc_test_51.schema_01;

In [0]:
%sql
-- 権原付与
GRANT USE CATALOG ON CATALOG uc_test_51 TO uc_test_all_read;
ALTER SCHEMA uc_test_51.schema_01 OWNER TO uc_test_all_read;

In [0]:
%sql
DESCRIBE SCHEMA uc_test_51.schema_01;

database_description_item,database_description_value
Catalog Name,uc_test_51
Namespace Name,schema_01
Comment,
Location,
Owner,uc_test_all_read


In [0]:
## S9 リソースの削除

In [0]:
%sql
DROP CATALOG uc_test_01 CASCADE;

In [0]:
%sql
DROP CATALOG uc_test_02 CASCADE;

In [0]:
%sql
DROP CATALOG uc_test_50 CASCADE;

In [0]:
%sql
DROP CATALOG uc_test_51 CASCADE;